## Задание 1

## Билет 52

In [1]:
from pulp import *
import pandas as pd

In [5]:
firms = ["A1", "A2", "A3"]
a1, a2, a3 = 60, 130, 90

supply = {"A1": a1, "A2": a2, "A3" : a3} 

consumers = ["B1", "B2", "B3", "B4"]
b1, b2, b3, b4 = 30, 80, 60, 110

demand = {
    "B1": b1,
    "B2": b2,
    "B3": b3,
    "B4": b4,
}

costs = [
    [6, 1000, 15, 4],  
    [9, 15, 2, 1000],  
    [6, 12, 7, 1] 
]

costs = makeDict([firms, consumers], costs, 0)

print(supply,"\n")

print(demand)

{'A1': 60, 'A2': 130, 'A3': 90} 

{'B1': 30, 'B2': 80, 'B3': 60, 'B4': 110}


#### Проверим, что транспортная задача закрытого типа

In [7]:
sum(supply.values()) == sum(demand.values())

True

Транспортная задача закрытого типа

#### Задача минимизации

In [8]:
model = LpProblem("Distribution_Problem", LpMinimize)
model

Distribution_Problem:
MINIMIZE
None
VARIABLES

#### Все возможные маршруты

In [10]:
Routes = [(w, b) for w in firms for b in consumers]
print(Routes)

[('A1', 'B1'), ('A1', 'B2'), ('A1', 'B3'), ('A1', 'B4'), ('A2', 'B1'), ('A2', 'B2'), ('A2', 'B3'), ('A2', 'B4'), ('A3', 'B1'), ('A3', 'B2'), ('A3', 'B3'), ('A3', 'B4')]


In [11]:
goods = LpVariable.dicts("Numbers_of_goods", (firms, consumers), 0, None, LpInteger)
goods

{'A1': {'B1': Numbers_of_goods_A1_B1,
  'B2': Numbers_of_goods_A1_B2,
  'B3': Numbers_of_goods_A1_B3,
  'B4': Numbers_of_goods_A1_B4},
 'A2': {'B1': Numbers_of_goods_A2_B1,
  'B2': Numbers_of_goods_A2_B2,
  'B3': Numbers_of_goods_A2_B3,
  'B4': Numbers_of_goods_A2_B4},
 'A3': {'B1': Numbers_of_goods_A3_B1,
  'B2': Numbers_of_goods_A3_B2,
  'B3': Numbers_of_goods_A3_B3,
  'B4': Numbers_of_goods_A3_B4}}

#### Целевая функция

In [12]:
model += (
    lpSum([goods[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs"
)
model

Distribution_Problem:
MINIMIZE
6*Numbers_of_goods_A1_B1 + 1000*Numbers_of_goods_A1_B2 + 15*Numbers_of_goods_A1_B3 + 4*Numbers_of_goods_A1_B4 + 9*Numbers_of_goods_A2_B1 + 15*Numbers_of_goods_A2_B2 + 2*Numbers_of_goods_A2_B3 + 1000*Numbers_of_goods_A2_B4 + 6*Numbers_of_goods_A3_B1 + 12*Numbers_of_goods_A3_B2 + 7*Numbers_of_goods_A3_B3 + 1*Numbers_of_goods_A3_B4 + 0
VARIABLES
0 <= Numbers_of_goods_A1_B1 Integer
0 <= Numbers_of_goods_A1_B2 Integer
0 <= Numbers_of_goods_A1_B3 Integer
0 <= Numbers_of_goods_A1_B4 Integer
0 <= Numbers_of_goods_A2_B1 Integer
0 <= Numbers_of_goods_A2_B2 Integer
0 <= Numbers_of_goods_A2_B3 Integer
0 <= Numbers_of_goods_A2_B4 Integer
0 <= Numbers_of_goods_A3_B1 Integer
0 <= Numbers_of_goods_A3_B2 Integer
0 <= Numbers_of_goods_A3_B3 Integer
0 <= Numbers_of_goods_A3_B4 Integer

#### Ограничения

In [13]:
for w in firms:
    model += (
        lpSum([goods[w][b] for b in consumers]) <= supply[w],
        "Sum_of_Products_out_of_firm_%s" % w
    )
    
for b in consumers:
    model += (
        lpSum([goods[w][b] for w in firms]) == demand[b],
        "Sum_of_Products_into_consumer%s" % b
    )
model   

Distribution_Problem:
MINIMIZE
6*Numbers_of_goods_A1_B1 + 1000*Numbers_of_goods_A1_B2 + 15*Numbers_of_goods_A1_B3 + 4*Numbers_of_goods_A1_B4 + 9*Numbers_of_goods_A2_B1 + 15*Numbers_of_goods_A2_B2 + 2*Numbers_of_goods_A2_B3 + 1000*Numbers_of_goods_A2_B4 + 6*Numbers_of_goods_A3_B1 + 12*Numbers_of_goods_A3_B2 + 7*Numbers_of_goods_A3_B3 + 1*Numbers_of_goods_A3_B4 + 0
SUBJECT TO
Sum_of_Products_out_of_firm_A1: Numbers_of_goods_A1_B1
 + Numbers_of_goods_A1_B2 + Numbers_of_goods_A1_B3 + Numbers_of_goods_A1_B4
 <= 60

Sum_of_Products_out_of_firm_A2: Numbers_of_goods_A2_B1
 + Numbers_of_goods_A2_B2 + Numbers_of_goods_A2_B3 + Numbers_of_goods_A2_B4
 <= 130

Sum_of_Products_out_of_firm_A3: Numbers_of_goods_A3_B1
 + Numbers_of_goods_A3_B2 + Numbers_of_goods_A3_B3 + Numbers_of_goods_A3_B4
 <= 90

Sum_of_Products_into_consumerB1: Numbers_of_goods_A1_B1
 + Numbers_of_goods_A2_B1 + Numbers_of_goods_A3_B1 = 30

Sum_of_Products_into_consumerB2: Numbers_of_goods_A1_B2
 + Numbers_of_goods_A2_B2 + Numbers_

In [14]:
model.solve()

1

In [15]:
print(f"Статус решения: {model.status}, {LpStatus[model.status]}\n")

for v in model.variables():
    print(v.name, "=", v.varValue)

print(f"\nОптимальное значение целевой функции = {value(model.objective)}")

Статус решения: 1, Optimal

Numbers_of_goods_A1_B1 = 30.0
Numbers_of_goods_A1_B2 = 0.0
Numbers_of_goods_A1_B3 = 0.0
Numbers_of_goods_A1_B4 = 30.0
Numbers_of_goods_A2_B1 = 0.0
Numbers_of_goods_A2_B2 = 70.0
Numbers_of_goods_A2_B3 = 60.0
Numbers_of_goods_A2_B4 = 0.0
Numbers_of_goods_A3_B1 = 0.0
Numbers_of_goods_A3_B2 = 10.0
Numbers_of_goods_A3_B3 = 0.0
Numbers_of_goods_A3_B4 = 80.0

Оптимальное значение целевой функции = 1670.0


#### Оптимальный план

In [17]:
totals = [v.varValue for v in model.variables()]

series_obj = pd.Series(totals)
  
arr = series_obj.values
  
reshaped_arr = arr.reshape((3, 4))
   
df = pd.DataFrame(reshaped_arr, index=firms, columns=consumers)

df

,B1,B2,B3,B4
A1,30.0,0.0,0.0,30.0
A2,0.0,70.0,60.0,0.0
A3,0.0,10.0,0.0,80.0


#### Оптимальная стоимость перевозки

In [18]:
print(f"Оптимальная стоимость перевозки = {int(value(model.objective))}")

Оптимальная стоимость перевозки = 1670
